<a href="https://colab.research.google.com/github/RiyaThakore/Question-generation-LSTM/blob/master/Question_generation(A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
  
df = getDF('/content/sample_data/qa_Appliances.json.gz')
#df = df.drop(['questionType', 'asin', 'answerTime', 'unixTime','answerType'], axis = 1)
a = df.to_csv('/content/sample_data/appliances.csv')
data = pd.read_csv("/content/sample_data/appliances.csv") 

def display_char_vocab(data):
  chars = sorted(list(set(data)))
  #char_to_int = dict((c, i) for i, c in enumerate(chars))
  n_chars = len(data)
  n_vocab = len(chars)
  return n_chars, n_vocab

seq_length = 100
#h = df.head()
#print(h)
#print(df.columns)
y = data.question
X = data.answer
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65,test_size=0.35)

def tokenize():
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(list(X_train))
  X_train_seq  = tokenizer.texts_to_sequences(X_train) 
  X_test_seq = tokenizer.texts_to_sequences(X_test)
  X_train_seq  = pad_sequences(X_train_seq, maxlen=100)
  X_test_seq = pad_sequences(X_test_seq, maxlen=100)
  y_train_seq  = tokenizer.texts_to_sequences(y_train) 
  y_test_seq = tokenizer.texts_to_sequences(y_test)
  y_train_seq  = pad_sequences(y_train_seq, maxlen=100)
  y_test_seq = pad_sequences(y_test_seq, maxlen=100)
  return X_train_seq, X_test_seq, y_train_seq, y_test_seq

X_train_seq, X_test_seq, y_train_seq, y_test_seq = tokenize()

#print("X_train_seq", X_train_seq)
model = Sequential()
model.add(LSTM(256, input_shape=(X_train_seq.shape[0], X_train_seq.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(100, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

lb = LabelEncoder()
predictions = model.predict(X_train_seq)
#print(predictions)
#print(y_train_seq)
y1=lb.inverse_transform(predictions)
#print(y1)

In [ ]:
import pandas as pd
import gzip
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
  
df = getDF('/content/sample_data/qa_Appliances.json.gz')
#df = df.drop(['questionType', 'asin', 'answerTime', 'unixTime','answerType'], axis = 1)
a = df.to_csv('/content/sample_data/appliances.csv')
data = pd.read_csv("/content/sample_data/appliances.csv") 

def display_char_vocab(data):
  chars = sorted(list(set(data)))
  char_to_int = dict((c, i) for i, c in enumerate(chars))
  n_chars = len(data)
  n_vocab = len(chars)
  return n_chars, n_vocab

seq_length = 100
#h = df.head()
#print(h)
#print(df.columns)
y = data.question
X = data.answer
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65,test_size=0.35)

def tokenize():
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(list(X_train))
  X_train_seq  = tokenizer.texts_to_sequences(X_train) 
  X_test_seq = tokenizer.texts_to_sequences(X_test)
  X_train_seq  = pad_sequences(X_train_seq, maxlen=100)
  X_test_seq = pad_sequences(X_test_seq, maxlen=100)
  y_train_seq  = tokenizer.texts_to_sequences(y_train) 
  y_test_seq = tokenizer.texts_to_sequences(y_test)
  y_train_seq  = pad_sequences(y_train_seq, maxlen=100)
  y_test_seq = pad_sequences(y_test_seq, maxlen=100)
  return X_train_seq, X_test_seq, y_train_seq, y_test_seq

print(X_train.shape)

X_train_seq, X_test_seq, y_train_seq, y_test_seq = tokenize()
#print("X_train_seq", X_train_seq)
model = Sequential()
model.add(LSTM(256, input_shape=(X_train_seq.shape), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(100, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

lb = LabelEncoder()
predictions = model.predict(X_test_seq)
#print(predictions)
#print(y_train_seq)
y1=lb.inverse_transform(predictions)
#print(y1)

In [ ]:
print(df.head)
print(len(df['question']))

<bound method NDFrame.head of                                                question                                             answer
0     I have a 9 year old Badger 1 that needs replac...   I replaced my old one with this without a hitch.
1                                          model number  This may help InSinkErator Model BADGER-1: Bad...
2     can I replace Badger 1 1/3 with a Badger 5 1/2...  Plumbing connections will vary with different ...
3     Does this come with power cord and dishwasher ...  It does not come with a power cord. It does co...
4     loud noise inside when turned on. sounds like ...  Check if you dropped something inside.Usually ...
...                                                 ...                                                ...
9006         How come it is compatible with all brands?  The Woder 5K Inline Water Filter for Ice ice m...
9007  Is it compatible to replace my Maytag UKF8001 ...  Yes, of course. The Woder Fridge Filter fits b...
9008  I